# Google Smart Home 

> Set up a custom smart home action to control some devices that require composite actions.

In [21]:
# | default_exp google
# | export
import uvicorn

from typing import List, Optional
from fastapi import FastAPI
from fastcore.script import call_parse
from pydantic import BaseModel, Field, conlist
from uuid import UUID, uuid4

## Motivation

* I have a ceiling fan with a light that can be controlled by two things:
    * A light switch actuated by a [Switchbot](https://www.switch-bot.com/), and
    * the ceiling fan itself controlled by a [Tuya RF Controller](https://expo.tuya.com/product/854009)
* I want to be able to control the light and fan from Google Assistant.


## Account Linking

* Set up the necessary project in Google cloud to create a smart home action 
    * **Warning:** If you create a smart home action on an existing project, then delete the project from the Actions Console, you will delete the GCP project as well. Thankfully, things can be restored!|
* I had set up terraform to point to port 3838 on my development server, so that I can create a Smart Home Action.
    * Nifty that you can reference a github address as a module, so I just took the module from [TJCloud](https://github.com/tjpalanca/tjcloud)
* Account linking had worked after a few rejiggers of terraform!

## Intent Fulfillment 

* The way that the Smart Home Actions work is by fulfilling "intents" that are sent to the endpoint.
* I couldn't get Google to reach my app through OAuth2 Proxy at first, it kept redirecting the sync request to login. Setting `--skip-jwt-bearer-token=true` fixed this as it allowed the request to come through with just the bearer token.

## Devices

In [22]:
# |export
class GoogleDevice(BaseModel):
    id: str
    custom_data: Optional[dict]

TypeError: Cannot instantiate typing.Optional

## Intents

In [47]:
# | export


class Intent(BaseModel):
    """
    Smart home intents are simple messaging objects that describe what smart home Action
    to perform such as turn on a light or cast audio to a speaker.

    [Reference](https://developers.google.com/assistant/smarthome/concepts/intents)
    """

    class Input(BaseModel):
        intent: str

        class Payload(BaseModel):
            devices: Optional[List[GoogleDevice]]

    request_id: UUID = Field(default_factory=uuid4)
    inputs: List[Input] = Field(min_items=1, max_items=1)


class SyncIntent(Intent):
    """
    This intent requests the list of devices associated with the given user and their
    capabilities. It is triggered during account linking or when a user manually resyncs
    their devices.

    [Reference](https://developers.google.com/assistant/smarthome/reference/intent/sync)
    """

    class SyncInput(Intent.Input):
        """Input for a sync intent"""

        intent: str = "action.devices.SYNC"

    inputs: List[SyncInput] = Field(min_items=1, max_items=1)


class QueryIntent(Intent):
    """
    This intent queries your fulfillment for the current states of devices, including
    whether the device is online and reachable. Should only retrun state information

    [Reference](https://developers.google.com/assistant/smarthome/reference/intent/query)
    """

    class QueryInput(Intent.Input):
        """Input for a query intent"""

        class QueryPayload(Intent.Input.Payload):
            devices: List[GoogleDevice]
            pass

        intent: str = "action.devices.QUERY"
        payload: QueryPayload

    inputs: List[QueryInput] = Field(min_items=1, max_items=1)


class ExecuteIntent(Intent):
    """
    This intent requests that your fulfillment execute a command for a device. This
    intent is triggered when a user says a phrase that matches a command defined in the
    manifest.

    [Reference](https://developers.google.com/assistant/smarthome/reference/intent/execute)
    """

    class ExecuteInput(Intent.Input):
        """Input for an execute intent"""

        class ExecutePayload(Intent.Input.Payload):
            class Command(BaseModel):
                class Execution(BaseModel):
                    command: str
                    params: Optional[dict]

                devices: List[GoogleDevice]
                execution: List[Execution]

            commands: List[Command]

        intent: str = "action.devices.EXECUTE"
        payload: ExecutePayload

    inputs: List[ExecuteInput] = Field(min_items=1, max_items=1)


class DisconnectIntent(Intent):
    """
    This intent requests that your fulfillment disconnect a user from your service. This
    intent indicates that Google Assistant will not send additional intents for this
    user.

    [Reference](https://developers.google.com/assistant/smarthome/reference/intent/disconnect)
    """

    class DisconnectInput(Intent.Input):
        """Input for a disconnect intent"""

        intent: str = "action.devices.DISCONNECT"

    inputs: List[DisconnectInput] = Field(min_items=1, max_items=1)

'{"request_id": "8e9173ff-6956-4b42-8292-a04a18a6bda4", "inputs": [{"intent": "action.devices.DISCONNECT"}]}'

In [ ]:
# | export
app = FastAPI()

In [ ]:
# | export
@app.post("/google/fulfillment")
def google_fulfillment():
    pass

In [ ]:
# | export


@call_parse
def run(reload=True):
    uvicorn.run(f"{__name__}:app", host="0.0.0.0", port=3838, reload=reload)

### Request Sync

* We can trigger a sync request using [Request SYNC](https://developers.google.com/assistant/smarthome/develop/request-sync#http).